In [ ]:
# %load_ext lab_black
import json
import os
import sys
import glob
from time import sleep

from  pythagoras import PolyphonicPlayer

import numpy as np
from scipy import signal
from scipy.io import wavfile as wav
import matplotlib.pyplot as plt

import plotly.graph_objects as go
from ipywidgets import interact, IntSlider

plt.style.use("dark_background")
base_layout = dict(
    template="plotly_dark",
    xaxis_showgrid=False,
    yaxis_showgrid=False,
    margin=dict(l=20, r=20, t=20, b=20),
)

In [ ]:
def get_peaks(amplitudes, height=400, distance=10):
    amplitudes = np.abs(amplitudes)
    peaks, peaks_prop = signal.find_peaks(amplitudes, height=height, distance=distance)
    heights = amplitudes[peaks]
    return peaks, heights


def save_peaks_and_volumes(peaks, volumes, name):
    npy_filename = os.path.join("chord_data", f"{name}.npy")
    with open(npy_filename, "wb") as f:
        np.save(f, (peaks, volumes))
        
        
def find_amplitudes(filename, lowest_frequency=30, octaves=7, bins_per_semitone=2):
    # TODO add caching
    rate, data = wav.read(filename)
    data = data[:, 0] #+ data[:, 1]  # convert to mono

    # note: resolution of the human ear is about 6 cents = 1/16 of a semitone
    delta_f = 2 ** (1/12/bins_per_semitone) - 1
    # based on https://dsp.stackexchange.com/questions/15574/morlet-wavelet-time-and-frequency-resolution
    # but not sure about the calculation
    omega = 1 / (np.sqrt(2) * delta_f)

    frequencies = lowest_frequency * 2 ** np.arange(0, octaves, 1/12/bins_per_semitone)
    widths = omega * rate / (2 * np.pi * frequencies)
    wavelet_spectrum = signal.cwt(data, signal.morlet2, widths, w=omega)

    wavelet_spectrum_abs = np.abs(wavelet_spectrum)
    amplitudes = wavelet_spectrum_abs.mean(axis=1)
    
    return amplitudes, frequencies

In [ ]:
files = glob.glob("data/*.wav")
files = sorted(files, key=os.path.getmtime, reverse=True)

fig = go.FigureWidget(layout=base_layout)
fig.update_layout(
    # xaxis_range=[0, len(total_means)],
    # yaxis_range=[0, max(total_means)],
    width=1500,
    height=300,
)
fig.add_scattergl()


player = PolyphonicPlayer(base_freq=1, max_voices=200)
player.start()

last_file = None

@interact(file=files, height=(0, 100000), distance=(1, 30), bins_per_semitone=(1, 16), play=False)
def update(file=None, height=0, distance=3, bins_per_semitone=2, play=False):
    global peaks, volumes, last_file, amplitudes, frequencies
    
    if file is None:
        return
    
    if last_file != file:
        amplitudes, frequencies = find_amplitudes(file, bins_per_semitone=bins_per_semitone)
        last_file = file
        
    peaks, volumes = get_peaks(amplitudes, height=height, distance=distance)
    
    player.ratios = frequencies[peaks]
    if play:
        player.volumes = volumes / np.sum(volumes)
    else:
        player.volumes = np.zeros(len(player.volumes))

    shapes = list()
    for peak, vol in zip(peaks, volumes):
        shapes.append(
            {
                "type": "line",
                "line_color": "orange",
                "x0": peak,
                "y0": 0,
                "x1": peak,
                "y1": vol,
            }
        )

    with fig.batch_update():
        fig.data[0].y = np.abs(amplitudes)
        fig.layout.shapes = shapes
        fig.update_layout(
            xaxis_range=[0, len(amplitudes)],
            yaxis_range=[0, max(amplitudes)],
        )
    print(len(peaks))


fig

In [ ]:
player.kill()
player.join()

In [ ]:
name = os.path.split(filename)[1].split(".")[0]
save_peaks_and_volumes(frequencies[peaks], volumes, name)

In [ ]:
# player.alive